# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

In [235]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

## The *Nim* and *Nimply* classes

In [236]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [237]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [238]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [239]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [240]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [241]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [242]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

## Task 3.1: Expert System

In [243]:
def solution(state: Nim):
    #It works really well with k = None
    x = nim_sum(state)
    next_state = [r ^ x for r in state.rows ]
    _min = next_state.index(min(next_state))
    if state.k:
        return _min, min(state.rows[_min] - next_state[_min], state.k-1)
    else:
        return _min, state.rows[_min] - next_state[_min]

def the_optimal_destroyer(state: Nim) -> Nimply:
    _row = 0
    _objects = 0
    if state.k:
        k_rows = [(i,(r%(state.k+1))) for i,r in enumerate(state.rows) if r % (state.k + 1)]
        if k_rows:
            _min = k_rows.index(min(k_rows, key = lambda a: a[1]))
            _row = k_rows[_min][0]
            _objects = min(k_rows[_min][1], state.k)
        else:
            _row, _objects = solution(state)
    else: _row, _objects = solution(state)
    return Nimply(_row, _objects)




## Evaluation of the Agent

In [244]:
NUM_MATCHES = 10
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_strategy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

In [245]:
evaluate(the_optimal_destroyer)

1.0

## Oversimplified match

In [249]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (optimal_strategy, the_optimal_destroyer)

nim = Nim(11,5)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 7 11 13 15 17 19 21>
DEBUG:root:status: After player 1 -> <0 3 5 7 7 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <0 3 5 7 7 11 13 15 17 15 21>
DEBUG:root:status: After player 1 -> <0 3 5 6 7 11 13 15 17 15 21>
DEBUG:root:status: After player 0 -> <0 3 0 6 7 11 13 15 17 15 21>
DEBUG:root:status: After player 1 -> <0 3 0 6 6 11 13 15 17 15 21>
DEBUG:root:status: After player 0 -> <0 2 0 6 6 11 13 15 17 15 21>
DEBUG:root:status: After player 1 -> <0 2 0 6 6 11 12 15 17 15 21>
DEBUG:root:status: After player 0 -> <0 2 0 6 6 10 12 15 17 15 21>
DEBUG:root:status: After player 1 -> <0 0 0 6 6 10 12 15 17 15 21>
DEBUG:root:status: After player 0 -> <0 0 0 4 6 10 12 15 17 15 21>
DEBUG:root:status: After player 1 -> <0 0 0 4 6 10 12 12 17 15 21>
DEBUG:root:status: After player 0 -> <0 0 0 4 5 10 12 12 17 15 21>
DEBUG:root:status: After player 1 -> <0 0 0 4 5 10 12 12 17 12